In [1]:
from collections import defaultdict

from pathlib import Path
import os
import sys
import json
import pickle
#os.environ['CUDA_VISIBLE_DEVICES']='0'  # don't occupy GPU, inference can work on CPU
import torch
import logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from typing import List, Dict, Tuple

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from vectorizers.bert_boris.bert_flair_from_notebook_vectorizer import BertFlairFromNotebookVectorizer, test
from flair.embeddings import BertEmbeddings
from flair.data import Sentence
from sklearn.pipeline import Pipeline

from python_ev.dataset import load_parts, load_df
from vectorizers.bert_boris.bert_base_spwsi import BertBaseSPWSI
from subtask1_solutions.base_solution import Subtask1BaseVectorizerFactory
import flair

02/07/2019 16:23:27 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English
/mnt/raid/v.omelchenko/anaconda/envs/rejection_clean/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
vectorizer = BertFlairFromNotebookVectorizer(tokens_field='FORM',
                                             model='bert-base-multilingual-cased', 
                                             add_bias=True, 
                                             normalize_out_embs=True, 
                                             pattern='_',
                                             prediction_cutoff=20,
                                             lemmatize_type=None,
                                             verbose=True, 
                                             exclude_lemma=False)

Loading bert pretrained: bert-base-multilingual-cased
Using torch device:  cuda
Loading bert done
Hyperparameters:
Add_bias: True
Normalize_out_embs: True
Pattern: _



In [3]:
def gen_list_of_inputs(sent, tok = 'MASK', lemma = ''):
    tokens = sent.split(' ')
    res = []
    for i in range(0, len(tokens) + 1):
        tokens = sent.split(' ')
        tokens.insert(i, tok)
        res.append({str(i):(tokens, i, lemma)})
    return res

In [155]:
def remove_punct(lst):
    return [elem for elem in lst if elem.isalpha()]

def get_neighbors(sent, num):
    if (num == 0):
        return (sent[0], '')
    elif (num == len(sent)):
        return (sent[len(sent)-1], '')
    else:
        return (sent[num-1], sent[num])

def get_dict_part_by_index(dct, index): # забрать часть словаря, соответствующую указанному предложению
    return {key:value for key, value in dct.items() if key.startswith(str(index)+'_')}

class BertSolver():
    
    
    def __init__(self, bert_model, train_csv, valid_csv):
        self.bert_model = bert_model
        self.train_data = train_csv
        self.valid_data = valid_csv
        self.train = self.train_data['text']
        self.valid = self.valid_data['text']
    
    def get_train_bert_inputs(self, tok = 'MASK', lemma = ''):
        res = {}
        for (sent_num, sentence) in enumerate(self.train):
            tokens = sentence.split(' ')
            for token_index in range(0, len(tokens) + 1):
                tokens = sentence.split(' ')
                tokens.insert(token_index, tok)
                res['{0}_{1}'.format(sent_num, token_index)] = (tokens, token_index, lemma)
        self.train_bert_inputs = res
    
    def get_valid_bert_inputs(self, tok = 'MASK', lemma = ''):
        res = {}
        for (sent_num, sentence) in enumerate(self.test):
            tokens = sentence.split(' ')
            for token_index in range(0, len(tokens) + 1):
                tokens = sentence.split(' ')
                tokens.insert(token_index, tok)
                res['{0}_{1}'.format(sent_num, token_index)] = (tokens, token_index, lemma)
        self.valid_bert_inputs = res
        
    def vectorize_train(self):
        self.vectorized_train = {key:value[0].split(' ') for key, value in self.bert_model.predict_sent_substitute_representatives(self.train_bert_inputs).items()}
        
    def vectorize_valid(self):
        self.vectorized_valid = {key:value[0].split(' ') for key, value in self.bert_model.predict_sent_substitute_representatives(self.valid_bert_inputs).items()}
        
    def remove_punct_train(self):
        self.vectorized_train = {key:remove_punct(value) for key, value in self.vectorized_train.items()}
    
    def remove_punct_valid(self):
        self.vectorized_valid = {key:remove_punct(value) for key, value in self.vectorized_valid.items()}
        
    def remove_neighbors_train(self):
        for sent_num, sent in enumerate(self.train):
            sent_tok = sent.split(' ')
            vect_sent = get_dict_part_by_index(self.vectorized_train, sent_num)
            for mask_num, example in enumerate(vect_sent):
                w1, w2 = get_neighbors(sent_tok, mask_num)
                try:
                    self.vectorized_train['{0}_{1}'.format(sent_num, mask_num)].remove(w1)
                except ValueError:
                    pass
                try:
                    self.vectorized_train['{0}_{1}'.format(sent_num, mask_num)].remove(w2)
                except ValueError:
                    pass
                
    def remove_neighbors_valid(self):
        for sent_num, sent in enumerate(self.valid):
            sent_tok = sent.split(' ')
            vect_sent = get_dict_part_by_index(self.vectorized_valid, sent_num)
            for mask_num, example in enumerate(vect_sent):
                w1, w2 = get_neighbors(sent_tok, mask_num)
                try:
                    self.vectorized_valid['{0}_{1}'.format(sent_num, mask_num)].remove(w1)
                except ValueError:
                    pass
                try:
                    self.vectorized_valid['{0}_{1}'.format(sent_num, mask_num)].remove(w2)
                except ValueError:
                    pass
                
    #TODO: функция, убирающая союзы (но сначала точно убедиться, что в данных нет союзов-"гэпперов")
    #TODO: функция, убирающая самые частые слова

In [156]:
train = pd.read_csv('dialogue2019/AGRR-2019/train.csv', sep = '\t')
valid = pd.read_csv('dialogue2019/AGRR-2019/dev.csv', sep = '\t')

In [157]:
tmp = BertSolver(vectorizer, train[:3], valid[:3])

In [158]:
tmp.get_train_bert_inputs()

In [159]:
tmp.vectorize_train()



generate_vecs: 0it [00:00, ?it/s]


Vectorization:   0%|          | 0/89 [00:00<?, ?it/s]

generate_vecs: 3it [00:00, 23.03it/s]


Vectorization:   3%|▎         | 3/89 [00:00<00:03, 23.10it/s]

INPUT TO BERT: [CLS] | >>[MASK]<< | будучи | в | про | ##шлы | ##и | че | ##тве | ##рг | в | со | ##фии | , | он | назвал | себя | ' | на | ##по | ##лов | ##ину | во | ##сточно | ##ев | ##ро | ##пе | ##ице | ##м | ' | , | и | за | последние | пять | д | ##не | ##и | у | ##с | ##пел | принять | у | себя | лидер | ##ов | укр | ##аи | ##ны | , | пол | ##ь | ##ши | и | че | ##хи | ##и | [SEP]
INPUT TO BERT: [CLS] | будучи | >>[MASK]<< | в | про | ##шлы | ##и | че | ##тве | ##рг | в | со | ##фии | , | он | назвал | себя | ' | на | ##по | ##лов | ##ину | во | ##сточно | ##ев | ##ро | ##пе | ##ице | ##м | ' | , | и | за | последние | пять | д | ##не | ##и | у | ##с | ##пел | принять | у | себя | лидер | ##ов | укр | ##аи | ##ны | , | пол | ##ь | ##ши | и | че | ##хи | ##и | [SEP]
INPUT TO BERT: [CLS] | будучи | в | >>[MASK]<< | про | ##шлы | ##и | че | ##тве | ##рг | в | со | ##фии | , | он | назвал | себя | ' | на | ##по | ##лов | ##ину | во | ##сточно | ##ев | ##ро | ##пе | ##ице | ##м | ' 



generate_vecs: 7it [00:00, 25.75it/s]


Vectorization:   8%|▊         | 7/89 [00:00<00:03, 25.81it/s]

generate_vecs: 11it [00:00, 28.14it/s]


Vectorization:  12%|█▏        | 11/89 [00:00<00:02, 28.15it/s]

INPUT TO BERT: [CLS] | будучи | в | про | ##шлы | ##и | че | ##тве | ##рг | в | со | ##фии | , | >>[MASK]<< | он | назвал | себя | ' | на | ##по | ##лов | ##ину | во | ##сточно | ##ев | ##ро | ##пе | ##ице | ##м | ' | , | и | за | последние | пять | д | ##не | ##и | у | ##с | ##пел | принять | у | себя | лидер | ##ов | укр | ##аи | ##ны | , | пол | ##ь | ##ши | и | че | ##хи | ##и | [SEP]
INPUT TO BERT: [CLS] | будучи | в | про | ##шлы | ##и | че | ##тве | ##рг | в | со | ##фии | , | он | >>[MASK]<< | назвал | себя | ' | на | ##по | ##лов | ##ину | во | ##сточно | ##ев | ##ро | ##пе | ##ице | ##м | ' | , | и | за | последние | пять | д | ##не | ##и | у | ##с | ##пел | принять | у | себя | лидер | ##ов | укр | ##аи | ##ны | , | пол | ##ь | ##ши | и | че | ##хи | ##и | [SEP]
INPUT TO BERT: [CLS] | будучи | в | про | ##шлы | ##и | че | ##тве | ##рг | в | со | ##фии | , | он | назвал | >>[MASK]<< | себя | ' | на | ##по | ##лов | ##ину | во | ##сточно | ##ев | ##ро | ##пе | ##ице | ##м | ' 



generate_vecs: 15it [00:00, 30.25it/s]


Vectorization:  17%|█▋        | 15/89 [00:00<00:02, 30.31it/s]

generate_vecs: 19it [00:00, 32.19it/s]


Vectorization:  21%|██▏       | 19/89 [00:00<00:02, 32.24it/s]

...



generate_vecs: 80it [00:01, 44.36it/s]


Vectorization:  90%|████████▉ | 80/89 [00:01<00:00, 44.36it/s]

generate_vecs: 85it [00:02, 44.70it/s]


Vectorization:  96%|█████████▌| 85/89 [00:02<00:00, 44.70it/s]

INPUT TO BERT: [CLS] | заместитель | генерального | секретаря | под | ##черк | ##нул | , | что | проблема | нар | ##кот | ##иков | не | только | де | ##ста | ##били | ##зи | ##рует | об | ##стан | ##ов | ##ку | в | стране | , | но | и | представляет | со | ##бо | ##и | у | ##гр | ##оз | ##у | миру | и | ст | ##аби | ##льно | ##сти | во | всем | регионе | , | и | приз | ##вал | ок | ##азы | ##вать | >>[MASK]<< | между | ##нар | ##од | ##ную | помощь | , | чтобы | обе | ##с | ##пе | ##чить | ос | ##у | ##ществ | ##ление | опера | ##тивного | плана | , | состав | ##ленного | ю | ##но | ##д | ##к | и | правительство | ##м | [SEP]
INPUT TO BERT: [CLS] | заместитель | генерального | секретаря | под | ##черк | ##нул | , | что | проблема | нар | ##кот | ##иков | не | только | де | ##ста | ##били | ##зи | ##рует | об | ##стан | ##ов | ##ку | в | стране | , | но | и | представляет | со | ##бо | ##и | у | ##гр | ##оз | ##у | миру | и | ст | ##аби | ##льно | ##сти | во | всем | регионе | , | и | п




Vectorization: 100%|██████████| 89/89 [00:02<00:00, 40.78it/s]

generate_vecs: 89it [00:02, 40.69it/s]

predict_word_scores:   0%|          | 0/89 [00:00<?, ?it/s]

INPUT TO BERT: [CLS] | заместитель | генерального | секретаря | под | ##черк | ##нул | , | что | проблема | нар | ##кот | ##иков | не | только | де | ##ста | ##били | ##зи | ##рует | об | ##стан | ##ов | ##ку | в | стране | , | но | и | представляет | со | ##бо | ##и | у | ##гр | ##оз | ##у | миру | и | ст | ##аби | ##льно | ##сти | во | всем | регионе | , | и | приз | ##вал | ок | ##азы | ##вать | между | ##нар | ##од | ##ную | помощь | , | чтобы | обе | ##с | ##пе | ##чить | ос | ##у | ##ществ | ##ление | опера | ##тивного | плана | , | состав | ##ленного | ю | ##но | ##д | ##к | и | >>[MASK]<< | правительство | ##м | [SEP]
INPUT TO BERT: [CLS] | заместитель | генерального | секретаря | под | ##черк | ##нул | , | что | проблема | нар | ##кот | ##иков | не | только | де | ##ста | ##били | ##зи | ##рует | об | ##стан | ##ов | ##ку | в | стране | , | но | и | представляет | со | ##бо | ##и | у | ##гр | ##оз | ##у | миру | и | ст | ##аби | ##льно | ##сти | во | всем | регионе | , | и | п



predict_word_scores:   1%|          | 1/89 [00:00<00:18,  4.71it/s]

predict_word_scores:   2%|▏         | 2/89 [00:00<00:20,  4.30it/s]

predict_word_scores:   3%|▎         | 3/89 [00:00<00:20,  4.22it/s]

predict_word_scores:   4%|▍         | 4/89 [00:00<00:20,  4.19it/s]

predict_word_scores:   6%|▌         | 5/89 [00:01<00:19,  4.25it/s]

predict_word_scores:   7%|▋         | 6/89 [00:01<00:19,  4.35it/s]

predict_word_scores:   8%|▊         | 7/89 [00:01<00:18,  4.42it/s]

predict_word_scores:   9%|▉         | 8/89 [00:01<00:18,  4.48it/s]

predict_word_scores:  10%|█         | 9/89 [00:02<00:17,  4.53it/s]

predict_word_scores:  11%|█         | 10/89 [00:02<00:17,  4.57it/s]

predict_word_scores:  12%|█▏        | 11/89 [00:02<00:16,  4.59it/s]

predict_word_scores:  13%|█▎        | 12/89 [00:02<00:16,  4.61it/s]

predict_word_scores:  15%|█▍        | 13/89 [00:02<00:16,  4.58it/s]

predict_word_scores:  16%|█▌        | 14/89 [00:03<00:16,  4.62it/s]

predict_word_scores:  17%|█

In [160]:
tmp.remove_punct_train()

In [161]:
tmp.remove_neighbors_train()

Будучи 
Будучи в
в прошлый
прошлый четверг
четверг в
в Софии,
Софии, он
он назвал
назвал себя
себя 'наполовину
'наполовину восточноевропейцем',
восточноевропейцем', и
и за
за последние
последние пять
пять дней
дней успел
успел принять
принять у
у себя
себя лидеров
лидеров Украины,
Украины, Польши
Польши и
и Чехии
Чехии 
Работа 
Работа с
с двухбайтовыми
двухбайтовыми наборами
наборами символов
символов —
— просто
просто кошмар
кошмар для
для программиста,
программиста, так
так как
как часть
часть их
их состоит
состоит из
из одного
одного байта,
байта, а
а часть
часть —
— из
из двух.
двух. 
Заместитель 
Заместитель Генерального
Генерального секретаря
секретаря подчеркнул,
подчеркнул, что
что проблема
проблема наркотиков
наркотиков не
не только
только дестабилизирует
дестабилизирует обстановку
обстановку в
в стране,
стране, но
но и
и представляет
представляет собой
собой угрозу
угрозу миру
миру и
и стабильности
стабильности во
во всем
всем регионе,
регионе, и
и призвал
призвал оказывать
о

In [162]:
tmp.vectorized_train

{'0_0': ['и',
  'А',
  'Уже',
  'И',
  'не',
  'Однако',
  'даже',
  'уже',
  'ещё',
  'Не',
  'В'],
 '0_1': ['уже',
  'и',
  'ещё',
  'только',
  'также',
  'на',
  'даже',
  'не',
  'одним',
  'первым',
  'еще',
  'его'],
 '0_2': ['первый',
  'этот',
  'его',
  'последний',
  'самый',
  'свой',
  'один',
  'за',
  'третий',
  'последние',
  'первые',
  'на',
  'новый',
  'их',
  'че',
  'уже',
  'своё',
  'начале'],
 '0_3': ['пять',
  'шесть',
  'четыре',
  'на',
  'и',
  'три',
  'два',
  'за',
  'трёх',
  'годы',
  'последние',
  'восемь',
  'пяти',
  'первый',
  'четырёх'],
 '0_4': ['года', 'и'],
 '0_5': ['своей',
  'начале',
  'русской',
  'этой',
  'первой',
  'новой',
  'его',
  'истории',
  'самой',
  'рамках',
  'ходе',
  'жизни',
  'немецкой',
  'результате',
  'годы',
  'городской',
  'конце',
  'составе'],
 '0_6': ['и',
  'так',
  'то',
  'как',
  'сам',
  'где',
  'здесь',
  'тогда',
  'что',
  'в',
  'когда',
  'а',
  'вскоре',
  'там',
  'на'],
 '0_7': ['и',
  'не',
  '

In [168]:
train[train['class'] == 1]

,text,class,cV,cR1,cR2,V,R1,R2
1,"Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.",1,92:99,83:91,103:109,127:127,119:124,127:134
5,"Выворачивает наизнанку, запад превращает в юг, а север — в восток, меняет местами добро и зло, велит открыть своё сердце, ничего не требуя взамен.",1,30:40,24:29,41:45,57:57,49:54,57:65
7,"Текущие обязательства компании в 2009 г. сократились на 16% - до 135.3 млн. грн., долгосрочные обязательства уменьшились на 1.7% - до 71.8 млн. грн., а дебиторская задолженность - на 3% до 87.7 млн. грн.",1,109:120,82:108,121:148,180:180,152:177,180:202
8,"Первая конвенция была ратифицирована 80 процентами государств – членов МОТ, а вторая – 87 процентами этих государств.",1,22:36,0:16,37:74,87:87,78:84,87:116
14,"Но Христианин сказал Упрямому: — Нет, сосед, лучше ты последуй примеру Сговорчивого. Мы в самом деле там получим то, о чем я сейчас говорил, а сверх того — великую славу.",1,105:112,113:139,NaN,156:156,156:169,NaN
15,"Средний срок действия контрактов в 2000 году для консультантов увеличился на 16 дней, а для индивидуальных подрядчиков — на 15 дней.",1,63:73,45:62,74:84,121:121,88:118,121:131
20,"Однако за счет переориентации на другие рынки в целом импорт куриного мяса превысил показатели 2001 г. по стоимости - на 10,5%, по физическим объемам - на 1,4%.",1,75:83,103:115,118:126,152:152,128:149,152:159
22,"В наше время скромность принимают за высокомерие, а доброту за слабость .",1,24:33,13:23,34:48,60:60,52:59,60:71
30,"В 2005 г. в газопроводы ЕСГ поступило 699,7 млрд м3 газа (в 2001 г. - 621,2 млрд м3), таким образом, объемы транспортировки газа выросли на 12,6 %.",1,28:37,0:9,38:56,70:70,58:67,70:83
34,"Закон состоит в том, что все вещества, которые разлагаются батареей, являются в жидком состоянии проводниками, а в твердом состоянии — непроводниками электричества гальванической батареи (393, 394, 404, 407, 413, 505, 676, 679, 697 и др).",1,69:77,78:96,97:109,135:135,113:132,135:237
